In [ ]:
# function to convert utc to human readable time
def utc2time(ts):
    import datetime
    return list(map(int,datetime.datetime.utcfromtimestamp(ts).strftime('%Y %m %d %H').split()))

In [ ]:
# this function returns number of days in a month
def days_in_month(year, month):
    from calendar import monthrange
    return monthrange(year, month)[1]

In [ ]:
# creating a list of all the coordinates
def make_coord():
    coord=[]

    count = 1

    lat=31.39
    for i in range(10):
        lon=74.29
        for j in range(8):
            lon=round(lon,2)
            lat=round(lat,2)
            coord.append([count,[lat,lon]])
            count+=1
            # print(f"lat:{lat}, lon:{lon}")
            lon-=0.02
        lat-=0.02

    lat=31.29
    for i in range(5):
        lon=74.13
        for j in range(2):
            lon=round(lon,2)
            lat=round(lat,2)
            coord.append([count,[lat,lon]])
            count+=1
            # print(f"lat:{lat}, lon:{lon}")
            lon-=0.02
        lat-=0.02
    return coord

In [ ]:
def params(data,hour,index,lon,lat):
    # a function that returns a list of parameters against a given json
    dt=data['list'][hour]['dt']
    aqi=data['list'][hour]['main']['aqi']
    co=data['list'][hour]['components']['co']
    no=data['list'][hour]['components']['no']
    no2=data['list'][hour]['components']['no2']
    o3=data['list'][hour]['components']['o3']
    so2=data['list'][hour]['components']['so2']
    pm2_5=data['list'][hour]['components']['pm2_5']
    pm10=data['list'][hour]['components']['pm10']
    nh3=data['list'][hour]['components']['nh3']

    temp=utc2time(dt)

    return [temp[0],temp[1],temp[2],temp[3],index,lon,lat,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10]

In [ ]:
# function that returns dataframe of temperature coordinates for a day based on coordinates and timestamp
import requests
import pandas as pd
def temp_data(coord,ts):

    # getting number of days
    date=utc2time(ts)
    year=date[0]
    month=date[1]
    day=date[2]


    # ts is the timestamp
    # coord is the coordinates
    url = f"https://api.weatherbit.io/v2.0/history/hourly?lat={coord[0]}&lon={coord[1]}&start_date={year}-{month}-{day}%3A00&end_date={year}-{month}-{day}%3A24&key=5e76500a32524b8281b8b0ecd8822c19"
    r = requests.get(url = url)
    temp=r.json()
    arr=[]
    for i in range(24):
        arr.append([temp['data'][i]['app_temp'],temp['data'][i]['dewpt'],temp['data'][i]['weather']['description']])
    return pd.DataFrame(arr,columns=['temp','dewpt','condition'])

In [ ]:
# Function that returns a dataframe for pollution data for a day based on coordinates and timestamp
import requests
import pandas as pd
def poll_data(coord,ts,zone):
    
        # getting number of days
        date=utc2time(ts)
        year=date[0]
        month=date[1]
        day=date[2]
    
        # ts is the timestamp
        # coord is the coordinates
        url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={coord[0]}&lon={coord[1]}&start={ts}&end={ts+82800}&appid=8d9efe95caf3922348a458d615abeb8c"
        r = requests.get(url = url)
        data=r.json()
        arr=[]
        for i in range(24):
            arr.append(params(data,i,zone,coord[0],coord[1]))
        return pd.DataFrame(arr,columns=['year','month','day','hour','zone','lon','lat','aqi','co','no','no2','o3','so2','nh3','pm2_5','pm10'])

In [ ]:
# function that extracts data for a day and returns a dataframe
def get_data(coord,ts):
    sleep(1) # to avoid rate limit
    temp=temp_data(coord[1],ts)
    poll=poll_data(coord[1],ts,coord[0])
    return pd.concat([poll, temp], axis=1)

In [ ]:
'''
steps to do:
1. read timestamp from csv
2. get next day from timestamp
3. get number of days loop should be called
4. create coordinates
5. create a loop to get data for each coordinate
    - get pollution data for each day
    - get weather data for each day
    - combine both dataframes
6. combine dataframes in that specified order
7. append dataframe to csv
8. save updated timestamp to csv
'''

import pandas as pd
from time import sleep

# reading timestamp from txt file into a variable
try:
    with open('timestamp.txt','r') as f:
        timestamp=int(f.read())
# writing timestamp to txt file if not found
except:
    timestamp=1609459200 # for january 1, 2021
    with open('timestamp.txt','w') as f:
        f.write(str(timestamp))

# getting number of days loop should be called
days=days_in_month(utc2time(timestamp)[0],utc2time(timestamp)[1])
# creating coordinates
coord=make_coord()

# creating a loop to get data for each coordinate
for i in range(1): # loop for days
    temp = []
    for j in range(len(coord)): # loop for coordinates
        temp.append(get_data(coord[j],timestamp))

# adding headings to csv if not found
try:
    with open('data.csv','r') as f:
        pass
except:
    with open('data.csv','w') as f:
        f.write('year,month,day,hour,zone,lon,lat,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10,temp,dewpt,condition')

# combining dataframes in that specified order
pd.concat(temp, axis=0).sort_index(kind='merge').to_csv('data.csv', index=False, header=False, mode='a')

# writing updated timestamp to txt file
with open('timestamp.txt','w') as f:
    f.write(str(timestamp+86400))

In [ ]:
url = f"https://api.weatherbit.io/v2.0/history/hourly?lat=31.39&lon=74.29&start_date=2021-01-01%3A00&end_date=2021-12-31%3A24&key=2de09181eb414ef5934361dcea304b71"
r = requests.get(url = url)
temp=r.json()

In [ ]:
# normalizing data for each day into a dataframe
timestamp=1609459200
arr=[]
for i in range(len(temp['data'])):
    temp1=utc2time(timestamp)
    arr.append([temp1[0],temp1[1],temp1[2],temp1[3],temp['data'][i]['app_temp'],temp['data'][i]['dewpt'],temp['data'][i]['weather']['description']])
    timestamp+=3600
df=pd.DataFrame(arr,columns=['year','month','day','hour','temp','dewpt','condition'])

In [ ]:
df

In [ ]:
# getting data of a year from openweathermap
import requests
url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat=31.39&lon=74.29&start=1609459200&end=1640995200&appid=8d9efe95caf3922348a458d615abeb8c"
r = requests.get(url = url)
temp1=r.json()

In [ ]:
# normalizing data for each day into a dataframe
arr=[]
for i in range(len(temp1['list'])):
    arr.append(params(temp1,i,1,31.39,74.29))
df1=pd.DataFrame(arr,columns=['year','month','day','hour','zone','lon','lat','aqi','co','no','no2','o3','so2','nh3','pm2_5','pm10'])

In [ ]:
pd.concat([df1, df], axis=1).to_csv('year_data.csv', index=False, mode='a')

Implementation for a year

In [ ]:
import requests
import pandas as pd
df_arr=[]
coord=make_coord()
for i in coord:
    r=requests.get(f"https://api.weatherbit.io/v2.0/history/hourly?lat={i[1][0]}&lon={i[1][1]}&start_date=2021-01-01%3A00&end_date=2021-12-31%3A24&key=5e76500a32524b8281b8b0ecd8822c19")
    temp=r.json()
    r=requests.get(f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={i[1][0]}&lon={i[1][1]}&start=1609459200&end=1640995200&appid=8d9efe95caf3922348a458d615abeb8c")
    temp1=r.json()
    
    # normalizing temperature data
    timestamp=1609459200
    arr=[]
    for j in range(len(temp['data'])):
        temp2=utc2time(timestamp)
        arr.append([temp2[0],temp2[1],temp2[2],temp2[3],temp['data'][j]['app_temp'],temp['data'][j]['dewpt'],temp['data'][j]['weather']['description']])
        timestamp+=3600
    df=pd.DataFrame(arr,columns=['year','month','day','hour','temp','dewpt','condition'])

    # normalizing pollution data
    arr=[]
    for j in range(len(temp1['list'])):
        arr.append(params(temp1,j,i[0],i[1][0],i[1][1]))
    df1=pd.DataFrame(arr,columns=['year','month','day','hour','zone','lon','lat','aqi','co','no','no2','o3','so2','nh3','pm2_5','pm10'])

    # combining dataframes
    df_arr.append(pd.concat([df1, df], axis=1))

# combining all dataframes into one
df=pd.concat(df_arr, axis=0).sort_index(kind='merge')

# saving data to csv
df.to_csv('year_data.csv', index=False, mode='a')

In [ ]:
# reading a json file into dictionary
import json
with open('temperature.json') as f:
    data = json.load(f)

data

In [ ]:
# storing dictionery into json file
import json
with open('temperature1.json', 'w') as f:
    json.dump(data, f)

In [ ]:
url = f"https://api.weatherbit.io/v2.0/history/hourly?lat=31.39&lon=74.29&start_date=2021-01-01%3A00&end_date=2021-12-31%3A24&key=5e76500a32524b8281b8b0ecd8822c19"
url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat=31.39&lon=74.29&start=1609459200&end=1640995200&appid=8d9efe95caf3922348a458d615abeb8c"